In [1]:
import os
import pandas as pd
import numpy as np
import re

In [2]:
file_path = "2023_2024_AQI_dataset.csv"
data = pd.read_csv(file_path, low_memory = False)
sorted_data = data.sort_values(by = ['sitename', 'datacreationdate'])

In [3]:
# 臺東對應是知本（水試所）測站
spec_data = sorted_data[(sorted_data['sitename'].isin(['二林', '中壢', '臺東', '橋頭', '斗六', '新港', '新營', '永和', '三重', '線西', '大城', '南投', '左營']))]
#spec_data = sorted_data[(sorted_data['sitename'].isin(['二林', '中壢', '斗六', '三重', '新港', '新營']))]
#spec_data = sorted_data[(sorted_data['sitename'].isin(['三重']))]

In [4]:
spec_data = spec_data.drop(columns = ['longitude', 'latitude', 'siteid', 'unit', 'county', 'pollutant', 'status', 'windspeed', 'winddirec'])

In [5]:
target_value = "2023/11/13 10:00:00" 
target_indices = spec_data[spec_data['datacreationdate'] == target_value].index

spec_data = spec_data.drop(index=target_indices)

In [6]:
# 將 datacreationdate 轉為 datetime 格式
spec_data['datacreationdate'] = pd.to_datetime(spec_data['datacreationdate'])
spec_data_select = spec_data[spec_data['datacreationdate'].dt.month.isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])]

In [7]:
# 定義一個函數，處理每個 sitename 的數據
def process_group(group):
    # 建立完整的時間序列
    full_time_range = pd.date_range(start=group['datacreationdate'].min(), end=group['datacreationdate'].max(), freq='h')
    # 重設索引並對齊時間序列
    group = group.set_index('datacreationdate').reindex(full_time_range).reset_index()
    group.rename(columns={"index": "datacreationdate"}, inplace=True)
    # 補充 sitename 並插值
    group['sitename'] = group['sitename'].ffill()  # 填補 sitename
    return group

In [8]:
spec_data_select_full = spec_data_select.groupby('sitename', group_keys=False).apply(process_group).reset_index(drop=True)

C:\Users\george\AppData\Local\Temp\ipykernel_47004\2564501761.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  spec_data_select_full = spec_data_select.groupby('sitename', group_keys=False).apply(process_group).reset_index(drop=True)


In [9]:
for column in spec_data_select_full.columns:
    
    if column != "datacreationdate":  
        if spec_data_select_full[column].isna().any():
            spec_data_select_full[column] = spec_data_select_full[column].fillna(spec_data_select_full[column].rolling(window=168, min_periods=1, center=True).mean().round(1))
        columns_to_round_first = ['aqi', 'pm10', 'pm2.5', 'pm10_avg', 'so2_avg'] 
        for column in columns_to_round_first:
             spec_data_select_full[column] = spec_data_select_full[column].round().astype(float)

In [10]:
spec_data_select_full

,datacreationdate,sitename,aqi,so2,co,o3,o3_8hr,pm10,pm2.5,no2,nox,no,co_8hr,pm2.5_avg,pm10_avg,so2_avg
0,2023-01-01 00:00:00,三重,40.0,0.8,0.8,22.8,17.6,13.0,14.0,23.3,37.7,14.4,1.0,12.0,17.0,0.0
1,2023-01-01 01:00:00,三重,45.0,0.9,0.76,22.3,18.7,26.0,22.0,21.9,38.6,16.6,0.9,14.0,19.0,0.0
2,2023-01-01 02:00:00,三重,42.0,0.5,0.67,24.2,19.9,12.0,5.0,21.0,30.5,9.5,0.9,13.0,18.0,0.0
3,2023-01-01 03:00:00,三重,37.0,0.5,0.57,24.6,21.1,6.0,4.0,20.5,30.4,9.8,0.8,12.0,15.0,0.0
4,2023-01-01 04:00:00,三重,32.0,0.4,0.47,26.3,22.3,3.0,4.0,15.7,24.4,8.7,0.7,10.0,13.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199363,2024-09-30 19:00:00,臺東,24.0,0.9,0.12,26.8,26.6,8.0,3.0,2.0,3.2,1.1,0.1,2.5,8.0,0.0
199364,2024-09-30 20:00:00,臺東,24.0,1.1,0.12,27.1,26.4,13.0,1.0,1.8,2.7,0.9,0.1,2.0,8.0,0.0
199365,2024-09-30 21:00:00,臺東,24.0,1.1,0.11,27.5,26.6,10.0,2.0,2.2,3.0,0.7,0.1,2.2,8.0,0.0
199366,2024-09-30 22:00:00,臺東,24.0,1.0,0.11,27.7,26.9,4.0,1.0,1.6,2.3,0.7,0.1,2.1,8.0,0.0


In [11]:
spec_data_select_full.isna().mean()

datacreationdate    0.0
sitename            0.0
aqi                 0.0
so2                 0.0
co                  0.0
o3                  0.0
o3_8hr              0.0
pm10                0.0
pm2.5               0.0
no2                 0.0
nox                 0.0
no                  0.0
co_8hr              0.0
pm2.5_avg           0.0
pm10_avg            0.0
so2_avg             0.0
dtype: float64

In [12]:
base_dir = "Weather_History_Data"
num_date = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]


def merge_weather_data(aqi_df, base_dir):
    for sitename in os.listdir(base_dir):
        sitename_path = os.path.join(base_dir, sitename)
        if os.path.isdir(sitename_path):
            for csv_file in os.listdir(sitename_path):
                if csv_file.endswith(".csv"):
                    # 提取日期，假設檔名格式為 `C0AI30-2024-01-01.csv`
                    split = csv_file.split("-")
                    year_str, month_str, date_str = split[1], split[2], split[3].replace(".csv", "")

                    csv_path = os.path.join(sitename_path, csv_file)
                    weather_df = pd.read_csv(csv_path)
                    weather_df = weather_df.drop(index=0).reset_index(drop=True)

                    target_columns = ["最大瞬間風(m/s)", "最大瞬間風風向(360degree)"] 
                    for column in target_columns:
                        if column in weather_df.columns:
                            weather_df = weather_df.drop(columns=[column])
                                        
                    weather_df['觀測時間(hour)'] = weather_df['觀測時間(hour)'].astype(int)
                    
                    for _, row in weather_df.iterrows():
                        # 構建 datacreationdate 的對應值
                        obs_time = row['觀測時間(hour)']
                        if obs_time != 24:
                            target_datetime = pd.to_datetime(f"{year_str}-{month_str}-{date_str} {obs_time:02}:00:00")
                        
                        # 處理 ObsTime 為 24 的情況，且為 2023 年，且非當月最後一天，轉到下一天
                        if obs_time == 24 and year_str == "2023" and date_str != num_date[int(month_str) - 1]:
                            target_datetime += pd.Timedelta(days=1)
                            target_datetime = target_datetime.replace(hour=0)
                        
                        # 處理 ObsTime 為 24 的情況，且為 2023 年，且為當月最後一天，轉到下個月第一天
                        if obs_time == 24 and year_str == "2023" and date_str == [int(month_str) - 1]:
                            target_datetime = target_datetime.replace(month=int(month_str) + 1, day=0, hour=0)

                        # 處理 ObsTime 為 24 的情況，且為 2023 年，且為當「年」最後一天，轉到 2024 年第一天
                        if obs_time == 24 and year_str == "2023" and date_str == [int(month_str) - 1] and month_str == "12":
                            target_datetime = target_datetime.replace(month=1, day=0, hour=0)
                        
                        # 處理 ObsTime 為 24 的情況，且為 2024 年，且非當月最後一天，轉到下一天
                        if obs_time == 24 and year_str == "2024" and date_str != num_date[int(month_str) - 1]:
                            target_datetime += pd.Timedelta(days=1)
                            target_datetime = target_datetime.replace(hour=0)

                        # 處理 ObsTime 為 24 的情況，且為 2024 年，且為當月最後一天，轉到下個月第一天
                        if obs_time == 24 and year_str == "2024" and date_str == num_date[int(month_str) - 1]:
                            target_datetime = target_datetime.replace(month=int(month_str) + 1, day=0, hour=0)

                        # 在 AQI 資料集中匹配 sitename 和 datacreationdate
                        mask = (aqi_df['sitename'] == sitename) & (aqi_df['datacreationdate'] == target_datetime)
                        for col in row.index:
                            if col not in ["觀測時間(hour)"]:
                                aqi_df.loc[mask, col] = row[col]

    return aqi_df


In [13]:
merge_data = merge_weather_data(spec_data_select_full, base_dir)

In [14]:
# 設定日期範圍
start_date = "2023-09-01 00:00:00"
end_date = "2024-09-01 00:00:00"

# 篩選在日期範圍內的資料
filtered_data = merge_data[
    (merge_data['datacreationdate'] >= start_date) & 
    (merge_data['datacreationdate'] <= end_date)
]

In [15]:
# 將各測站 2023-09-01 00:00:00 的資料刪除
filtered_data = filtered_data.groupby('sitename').apply(lambda group: group.iloc[1:]).reset_index(drop=True)

C:\Users\george\AppData\Local\Temp\ipykernel_47004\2036022196.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_data = filtered_data.groupby('sitename').apply(lambda group: group.iloc[1:]).reset_index(drop=True)


In [16]:
filtered_data

,datacreationdate,sitename,aqi,so2,co,o3,o3_8hr,pm10,pm2.5,no2,...,co_8hr,pm2.5_avg,pm10_avg,so2_avg,測站氣壓(hPa),氣溫(℃),相對溼度(%),風速(m/s),風向(360degree),降水量(mm)
0,2023-09-01 01:00:00,三重,27.0,0.9,0.6,8.6,8.8,15.0,8.0,13.3,...,0.6,7.6,15.0,0.0,1002.3,27.7,92,1.9,69,2.0
1,2023-09-01 02:00:00,三重,26.0,0.9,0.6,8.4,8.8,15.0,7.0,13.2,...,0.6,7.6,15.0,0.0,1001.1,28.0,90,3.4,57,0.0
2,2023-09-01 03:00:00,三重,26.0,0.8,0.6,8.2,8.7,15.0,7.0,13.2,...,0.6,7.5,15.0,0.0,1001.2,27.5,91,4.4,90,0.5
3,2023-09-01 04:00:00,三重,26.0,0.8,0.6,8.0,8.6,15.0,7.0,13.1,...,0.6,7.4,15.0,0.0,1001.3,27.2,91,5.4,73,0.0
4,2023-09-01 05:00:00,三重,26.0,0.8,0.6,7.9,8.5,15.0,7.0,13.0,...,0.6,7.4,15.0,0.0,1001.4,27.3,88,3.9,68,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114187,2024-08-31 20:00:00,臺東,19.0,1.6,0.15,16.2,21.1,11.0,2.0,5.1,...,0.1,4.5,17.0,1.0,1008.4,27.8,75,2.4,324,0.0
114188,2024-08-31 21:00:00,臺東,19.0,1.4,0.17,16.3,20.1,14.0,4.0,3.8,...,0.1,4.2,18.0,1.0,1008.9,27.8,75,0.8,307,0.0
114189,2024-08-31 22:00:00,臺東,19.0,1.6,0.17,15.0,18.9,17.0,3.0,7.0,...,0.1,3.9,19.0,1.0,1009.2,27.4,74,1.0,333,0.0
114190,2024-08-31 23:00:00,臺東,19.0,1.6,0.13,16.4,18.2,12.0,2.0,5.0,...,0.1,3.8,19.0,1.0,1009.3,27.4,72,1.3,316,0.0


In [ ]:
# 定義非數字的特殊字元正則表達式
special_char_pattern = re.compile(r"[^a-zA-Z0-9\s]")  # 匹配非數字、非字母、非空白字符

# 收集所有非數字的特殊字元並找出唯一值
unique_special_chars = set()  # 使用集合來存儲唯一的特殊字元

for column in filtered_data.columns:
    for value in filtered_data[column]:
        matches = special_char_pattern.findall(str(value))  # 尋找所有非數字特殊字元
        unique_special_chars.update(matches)  # 將找到的字元添加到集合中

{'&',
 '-',
 '.',
 '/',
 ':',
 '三',
 '中',
 '二',
 '六',
 '南',
 '和',
 '城',
 '壢',
 '大',
 '左',
 '投',
 '斗',
 '新',
 '東',
 '林',
 '橋',
 '永',
 '港',
 '營',
 '線',
 '臺',
 '西',
 '重',
 '頭'}

In [17]:
def replace_special_chars_with_nan(value):
    if re.search(r'--|&|\/|X', str(value)):  # 檢查是否包含 / 或 &
        return np.nan
    return value
filtered_data_cleaned = filtered_data.map(replace_special_chars_with_nan)
filtered_data_cleaned.isna().mean()

datacreationdate    0.000000
sitename            0.000000
aqi                 0.000000
so2                 0.000000
co                  0.000000
o3                  0.000000
o3_8hr              0.000000
pm10                0.000000
pm2.5               0.000000
no2                 0.000000
nox                 0.000000
no                  0.000000
co_8hr              0.000000
pm2.5_avg           0.000000
pm10_avg            0.000000
so2_avg             0.000000
測站氣壓(hPa)           0.017269
氣溫(℃)               0.017532
相對溼度(%)             0.017068
風速(m/s)             0.016192
風向(360degree)       0.016192
降水量(mm)             0.012996
dtype: float64

In [18]:
columns_to_convert = filtered_data_cleaned.columns[2:]  # 第三列及之後的欄位
filtered_data_cleaned[columns_to_convert] = filtered_data_cleaned[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [19]:
for column in columns_to_convert:
    filtered_data_cleaned[column] = filtered_data_cleaned[column].fillna(filtered_data_cleaned[column].rolling(window=168, min_periods=1, center=True).mean().round(1))

In [20]:
filtered_data_cleaned.isna().mean()

datacreationdate    0.0
sitename            0.0
aqi                 0.0
so2                 0.0
co                  0.0
o3                  0.0
o3_8hr              0.0
pm10                0.0
pm2.5               0.0
no2                 0.0
nox                 0.0
no                  0.0
co_8hr              0.0
pm2.5_avg           0.0
pm10_avg            0.0
so2_avg             0.0
測站氣壓(hPa)           0.0
氣溫(℃)               0.0
相對溼度(%)             0.0
風速(m/s)             0.0
風向(360degree)       0.0
降水量(mm)             0.0
dtype: float64

In [21]:
filtered_data_cleaned.dtypes

datacreationdate    datetime64[ns]
sitename                    object
aqi                        float64
so2                        float64
co                         float64
o3                         float64
o3_8hr                     float64
pm10                       float64
pm2.5                      float64
no2                        float64
nox                        float64
no                         float64
co_8hr                     float64
pm2.5_avg                  float64
pm10_avg                   float64
so2_avg                    float64
測站氣壓(hPa)                  float64
氣溫(℃)                      float64
相對溼度(%)                    float64
風速(m/s)                    float64
風向(360degree)              float64
降水量(mm)                    float64
dtype: object

In [22]:
filtered_data_cleaned

,datacreationdate,sitename,aqi,so2,co,o3,o3_8hr,pm10,pm2.5,no2,...,co_8hr,pm2.5_avg,pm10_avg,so2_avg,測站氣壓(hPa),氣溫(℃),相對溼度(%),風速(m/s),風向(360degree),降水量(mm)
0,2023-09-01 01:00:00,三重,27.0,0.9,0.60,8.6,8.8,15.0,8.0,13.3,...,0.6,7.6,15.0,0.0,1002.3,27.7,92.0,1.9,69.0,2.0
1,2023-09-01 02:00:00,三重,26.0,0.9,0.60,8.4,8.8,15.0,7.0,13.2,...,0.6,7.6,15.0,0.0,1001.1,28.0,90.0,3.4,57.0,0.0
2,2023-09-01 03:00:00,三重,26.0,0.8,0.60,8.2,8.7,15.0,7.0,13.2,...,0.6,7.5,15.0,0.0,1001.2,27.5,91.0,4.4,90.0,0.5
3,2023-09-01 04:00:00,三重,26.0,0.8,0.60,8.0,8.6,15.0,7.0,13.1,...,0.6,7.4,15.0,0.0,1001.3,27.2,91.0,5.4,73.0,0.0
4,2023-09-01 05:00:00,三重,26.0,0.8,0.60,7.9,8.5,15.0,7.0,13.0,...,0.6,7.4,15.0,0.0,1001.4,27.3,88.0,3.9,68.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114187,2024-08-31 20:00:00,臺東,19.0,1.6,0.15,16.2,21.1,11.0,2.0,5.1,...,0.1,4.5,17.0,1.0,1008.4,27.8,75.0,2.4,324.0,0.0
114188,2024-08-31 21:00:00,臺東,19.0,1.4,0.17,16.3,20.1,14.0,4.0,3.8,...,0.1,4.2,18.0,1.0,1008.9,27.8,75.0,0.8,307.0,0.0
114189,2024-08-31 22:00:00,臺東,19.0,1.6,0.17,15.0,18.9,17.0,3.0,7.0,...,0.1,3.9,19.0,1.0,1009.2,27.4,74.0,1.0,333.0,0.0
114190,2024-08-31 23:00:00,臺東,19.0,1.6,0.13,16.4,18.2,12.0,2.0,5.0,...,0.1,3.8,19.0,1.0,1009.3,27.4,72.0,1.3,316.0,0.0


In [23]:
# 將 DataFrame 輸出為 CSV 檔案
output_path = "2023_09_2024_08_AQI_Weather_dataset.csv"
filtered_data_cleaned.to_csv(output_path, index=False, encoding="utf-8")